In [9]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
import operator

load_dotenv()

True

In [ ]:
# create a model
model = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
# pydantic class for structured output
class EvaluationSchema(BaseModel):
    feedback: str = Field(description='Detailed feedback of the  essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

In [ ]:
# structured output model
structured_model = model.with_structured_output(EvaluationSchema)

In [ ]:
# essay variable
essay = """
Role of India in Artificial Intelligence

Artificial Intelligence (AI) is transforming the global landscape across industries such as healthcare, agriculture, education, transportation, and governance. As one of the world's most populous and rapidly developing nations, India is poised to play a pivotal role in shaping the future of AI, both within its borders and on the global stage. With a robust technology ecosystem, a vast talent pool, and increasing government support, India is emerging as a key player in the AI revolution.

1. India's Strategic Vision for AI

The Government of India has recognized AI as a strategic technology with the potential to drive economic growth and social inclusion. The National Strategy for Artificial Intelligence, launched by NITI Aayog in 2018, laid the foundation for AI development in the country. The strategy identifies five key sectors where AI can have the most significant impact: healthcare, agriculture, education, smart mobility, and smart cities. The vision is to position India not only as a consumer of AI technology but also as a global hub for AI development and deployment.

2. Growing AI Talent and Research

India produces one of the largest numbers of STEM (Science, Technology, Engineering, and Mathematics) graduates in the world. This vast talent pool is instrumental in fueling AI research and innovation. Leading Indian institutes like the Indian Institutes of Technology (IITs), Indian Institute of Science (IISc), and several private universities are investing in AI-focused research and offering specialized courses.

Additionally, India has seen a surge in AI research publications, patents, and startup activity. Collaborations between academia, industry, and government agencies are enhancing India's AI capabilities. Initiatives like the Centre for Artificial Intelligence and Robotics (CAIR) and partnerships with global tech firms further reinforce India’s R&D efforts.

3. AI Startups and Industry Growth

India has a rapidly growing AI startup ecosystem. Cities like Bangalore, Hyderabad, Pune, and Delhi are becoming hubs for AI-based innovation. These startups are working on solutions ranging from predictive healthcare diagnostics to precision agriculture, automated legal services, and language translation tools.

Major Indian IT companies such as TCS, Infosys, Wipro, and HCL are also heavily investing in AI, integrating it into their services and developing AI-powered platforms. Their global reach enables Indian AI talent and solutions to have a worldwide impact.

4. Government Initiatives and Policies

The Indian government is playing a proactive role in supporting AI adoption through funding, infrastructure, and policy frameworks. Programs like 'Responsible AI for Social Empowerment (RAISE)', AI for All, and Digital India aim to democratize access to AI and ensure its ethical and inclusive development.

Moreover, government-backed initiatives like IndiaAI and SAMARTH (Smart Automation and Advanced Manufacturing Hub) seek to foster AI innovation and application in public services and manufacturing. These initiatives are crucial in integrating AI into India’s socio-economic fabric.

5. Challenges and the Way Forward

Despite its progress, India faces challenges in realizing its full potential in AI. These include:

Data availability and quality: Reliable and annotated datasets are crucial for training AI models.

Skilling and education: While talent is abundant, there is a need for upskilling and re-skilling the workforce in AI technologies.

Ethical and regulatory frameworks: Ensuring responsible use of AI, data privacy, and transparency is vital.

To overcome these hurdles, India must invest in AI infrastructure, promote public-private partnerships, and formulate clear policies that encourage innovation while safeguarding public interest.

Conclusion

India stands at a crucial juncture in the AI revolution. With a clear strategic vision, strong talent base, vibrant startup ecosystem, and growing international collaborations, India is well-positioned to become a global leader in Artificial Intelligence. By focusing on inclusive and responsible AI development, India can leverage this transformative technology to address its unique challenges and contribute meaningfully to the global AI landscape.
"""

In [ ]:
# State of the UPSC candidate
class UPSCState(TypedDict):
    essay: str
    COT_feedback: str                # Clearity of thoughts
    DOA_feedback: str                # Depth of Analysis
    language_feedback: str           # Language
    final_feedback: str

    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [21]:
def evaluate_language(state: UPSCState):
    prompt = f'Evaluate the language quality of the following essay and provide feedback and assign a score out of 10 \n {state['essay']}'
    output = structured_model.invoke(prompt)
    return {'language_feedback':output.feedback, 'individual_scores':[output.score]}

In [22]:
def evaluate_analysis(state: UPSCState):
    prompt = f'Evaluate the depth of the following essay and provide feedback and assign a score out of 10 \n {state['essay']}'
    output = structured_model.invoke(prompt)
    return {'DOA_feedback':output.feedback, 'individual_scores':[output.score]}

In [30]:
def evaluate_thought(state: UPSCState):
    prompt = f'Evaluate the clarity of thought in the following essay and provide feedback and assign a score out of 10 \n {state['essay']}'
    output = structured_model.invoke(prompt)
    return {'COT_feedback':output.feedback, 'individual_scores':[output.score]}

In [27]:
def final_evaluation(state: UPSCState):
    # summary feedback
    prompt = f"""Based on the following feedbacks create a summarized feedback \n 
    language_feedback - {state['language_feedback']} \n 
    depth of analysis - {state['DOA_feedback']} \n 
    clarity of thought feedback - {state['COT_feedback']}"""
    overall_feedback = model.invoke(prompt).content
    
    # calculate average
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'final_feedback':overall_feedback, 'avg_score':avg_score}

In [32]:
# define graph
graph = StateGraph(UPSCState)

# add nodes
graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought',  evaluate_thought)
graph.add_node('final_evaluation',  final_evaluation)

# add edges
graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')

graph.add_edge('final_evaluation', END)

# compile
workflow = graph.compile()

In [33]:
initial_state = {
    'essay': essay,
}

final_state = workflow.invoke(initial_state)

In [34]:
final_state

{'essay': "\nRole of India in Artificial Intelligence\n\nArtificial Intelligence (AI) is transforming the global landscape across industries such as healthcare, agriculture, education, transportation, and governance. As one of the world's most populous and rapidly developing nations, India is poised to play a pivotal role in shaping the future of AI, both within its borders and on the global stage. With a robust technology ecosystem, a vast talent pool, and increasing government support, India is emerging as a key player in the AI revolution.\n\n1. India's Strategic Vision for AI\n\nThe Government of India has recognized AI as a strategic technology with the potential to drive economic growth and social inclusion. The National Strategy for Artificial Intelligence, launched by NITI Aayog in 2018, laid the foundation for AI development in the country. The strategy identifies five key sectors where AI can have the most significant impact: healthcare, agriculture, education, smart mobility